In [6]:
import neptune
import pandas as pd

In [7]:
session = neptune.sessions.Session()
project = session.get_project('andrzejzdobywca/pretrainingpp')

In [8]:
exps_ssl = project.get_experiments(tag='ssl_pretrained')
exps_clean = project.get_experiments(tag='clean_pretrained')
exps_imagenet_ssl = project.get_experiments(tag='ssl')
exps_imagenet_clean = project.get_experiments(tag='clean')

In [9]:
print(len(exps_ssl))
print(len(exps_clean))
print(len(exps_imagenet_ssl))
print(len(exps_imagenet_clean))

25
25
25
25


In [10]:
def get_best_acc(exps, name):
    my_array = []
    for exp in exps:
        res = exp.get_parameters()
        lr = res['lr']
        weightDecay = res['weightDecay']

        data_ssl = exp.get_numeric_channels_values('val_acc')
        best_acc = data_ssl['val_acc'].max()

        my_array.append([lr, weightDecay, best_acc])

    result = pd.DataFrame(my_array,columns=['lr', 'weightDecay', '{}'.format(name)])
    return result

In [11]:
res_ssl = get_best_acc(exps_ssl, 'cifar_ssl')
res_imagenet_ssl = get_best_acc(exps_imagenet_ssl, 'imagenet_ssl')
res_imagenet_clean = get_best_acc(exps_imagenet_clean, 'imagenet_clean')
res_clean = get_best_acc(exps_clean, 'cifar_clean')

In [12]:
print(res_imagenet_ssl)
print(res_imagenet_clean)

         lr  weightDecay  imagenet_ssl
0   0.01250     0.001000        0.0572
1   0.00250     0.001000        0.2362
2   0.00050     0.001000        0.3578
3   0.01250     0.000200        0.0584
4   0.00250     0.000200        0.3298
5   0.00050     0.000200        0.3224
6   0.01250     0.000040        0.1438
7   0.00250     0.000040        0.3598
8   0.00050     0.000040        0.3058
9   0.00250     0.000008        0.2780
10  0.01250     0.000008        0.2038
11  0.00050     0.000008        0.3014
12  0.00250     0.000002        0.2714
13  0.00050     0.000002        0.2994
14  0.01250     0.000002        0.1898
15  0.00010     0.001000        0.1938
16  0.00010     0.000200        0.1932
17  0.00010     0.000040        0.1984
18  0.00010     0.000008        0.2008
19  0.00010     0.000002        0.1824
20  0.00002     0.001000        0.0938
21  0.00002     0.000200        0.0988
22  0.00002     0.000040        0.0974
23  0.00002     0.000008        0.1024
24  0.00002     0.000002 

In [16]:
result = res_clean
for df in [res_ssl, res_imagenet_clean, res_imagenet_ssl]:
    result = result.merge(df, how='left', on=['lr', 'weightDecay'])
result = result.sort_values(by=['cifar_clean', 'cifar_ssl'])
print(result)

         lr  weightDecay  cifar_clean  cifar_ssl  imagenet_clean  imagenet_ssl
21  0.00002     0.000200       0.4211     0.4464          0.0930        0.0988
23  0.00002     0.000008       0.4230     0.4393          0.1030        0.1024
22  0.00002     0.000040       0.4255     0.4192          0.0952        0.0974
20  0.00002     0.001000       0.4394     0.4266          0.1144        0.0938
24  0.00002     0.000002       0.4408     0.4091          0.1082        0.0824
0   0.01250     0.001000       0.4689     0.4207          0.0366        0.0572
18  0.00010     0.000008       0.5601     0.5829          0.2002        0.2008
17  0.00010     0.000040       0.5679     0.5884          0.2036        0.1984
16  0.00010     0.000200       0.5755     0.5719          0.1922        0.1932
19  0.00010     0.000002       0.5792     0.5717          0.2082        0.1824
15  0.00010     0.001000       0.6077     0.5995          0.2102        0.1938
12  0.01250     0.000200       0.6475     0.5750    